In [1]:
import requests
import csv
import json

PLUTO_URL = (
    "https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/ArcGIS/rest/services/"
    "MAPPLUTO/FeatureServer/0/query"
)

In [2]:
def query_point(lat, lon, extra_fields=None):
    """
    Query MapPLUTO for a given lat/lon.
    Returns dict with bbl and assesstot, or indicates not assigned.
    """
    fields = ["bbl", "assesstot"] + (extra_fields or [])
    params = {
        "geometry": f"{lon},{lat}",
        "geometryType": "esriGeometryPoint",
        "inSR": "4326",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": ",".join(fields),
        "f": "json"
    }
    resp = requests.get(PLUTO_URL, params=params)
    resp.raise_for_status()
    res = resp.json()
    # print(f'response {res}')
    feats = res.get("features", [])
    print(f'feats {feats}')
    assessed_val = feats[0]["attributes"]#["AssessTot"]
    print(f'assessed_val {assessed_val}')
    if not feats:
        return {"BBL": None, "AssessTot": None, "status": "No tax value assigned"}
    attr = feats[0].get("attributes", {})
    return {
        "bbl": attr.get("BBL"),
        "assesstot": attr.get("AssessTot"),
        "status": ("Tax value assigned" if attr.get("AssessTot") is not None else "No tax value assigned")
    }

def batch_process(points, output_csv=None, output_geojson=None):
    results = []
    for lat, lon in points:
        info = query_point(lat, lon)
        info.update({"latitude": lat, "longitude": lon})
        results.append(info)

    if output_csv:
        with open(output_csv, "w", newline="") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=["latitude","longitude","bbl","assesstot","status"])
            writer.writeheader()
            writer.writerows(results)

    if output_geojson:
        geo = {"type": "FeatureCollection", "features": []}
        for rec in results:
            feat = {
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [rec["longitude"], rec["latitude"]]},
                "properties": {
                    "bbl": rec["bbl"],
                    "assesstot": rec["assesstot"],
                    "status": rec["status"]
                }
            }
            geo["features"].append(feat)
        with open(output_geojson, "w") as f:
            json.dump(geo, f, indent=2)

    return results

In [5]:
# Example usage
if __name__ == "__main__":
    test_points = [
        (40.7484, -73.9857), #Empire State bldg
        (40.60953705414596, -73.95857916073192), #TJ Maxx
        (40.610563308066425, -73.95488844147069), #khebab
        (40.610608084814885, -73.95444897008612), # sushi 
        (40.61492592654441, -73.95166846815303),
        (40.612132334704754, -73.96297976218243) #staples
    ]
    res = batch_process(test_points, output_csv="tax_lot_info.csv", output_geojson="tax_lots.geojson")
    print(res)

feats [{'attributes': {'BBL': 1008350041, 'AssessTot': 534044700}, 'geometry': {'rings': [[[987926.190470219, 211999.934650898], [987975.35999012, 212089.042743683], [988037.503561497, 212054.96932745], [988088.901752949, 212148.115113258], [988467.603769779, 211940.474148273], [988367.029474735, 211758.224841595], [987926.190470219, 211999.934650898]]]}}]
assessed_val {'BBL': 1008350041, 'AssessTot': 534044700}
feats [{'attributes': {'BBL': 3067770017, 'AssessTot': 18587700}, 'geometry': {'rings': [[[995653.927976131, 161400.437409401], [995753.440223694, 161415.271719933], [995787.561219692, 161178.77643919], [995766.277480602, 161175.603556156], [995746.696815014, 161172.684602737], [995726.828082561, 161169.722337246], [995708.903246403, 161167.050089359], [995687.750419617, 161163.897201061], [995587.353944778, 161148.930698872], [995570.184709549, 161266.816632271], [995671.195966721, 161281.874666214], [995665.254702091, 161322.667564392], [995564.243444443, 161307.609530449], [